<a href="https://colab.research.google.com/github/yhp2205/SQL/blob/main/ch_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 9강 시계열 기반으로 데이터 집계하기

In [1]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


9강에서는 데이터의 변화를 이해하기 쉽게 표현할 수 있는 리포팅 방법과 그에 대한 내용을 소개합니다.  
우선 샘플 데이터를 불러오겠습니다. 

In [2]:
%%bigquery --project mygcpproject-340112
DROP TABLE IF EXISTS sqldata.purchase_log;
CREATE TABLE sqldata.purchase_log(
    dt              string
  , order_id        integer
  , user_id         string
  , purchase_amount integer
);

INSERT INTO sqldata.purchase_log
VALUES
    ('2014-01-01',  1, 'rhwpvvitou', 13900)
  , ('2014-01-01',  2, 'hqnwoamzic', 10616)
  , ('2014-01-02',  3, 'tzlmqryunr', 21156)
  , ('2014-01-02',  4, 'wkmqqwbyai', 14893)
  , ('2014-01-03',  5, 'ciecbedwbq', 13054)
  , ('2014-01-03',  6, 'svgnbqsagx', 24384)
  , ('2014-01-03',  7, 'dfgqftdocu', 15591)
  , ('2014-01-04',  8, 'sbgqlzkvyn',  3025)
  , ('2014-01-04',  9, 'lbedmngbol', 24215)
  , ('2014-01-04', 10, 'itlvssbsgx',  2059)
  , ('2014-01-05', 11, 'jqcmmguhik',  4235)
  , ('2014-01-05', 12, 'jgotcrfeyn', 28013)
  , ('2014-01-05', 13, 'pgeojzoshx', 16008)
  , ('2014-01-06', 14, 'msjberhxnx',  1980)
  , ('2014-01-06', 15, 'tlhbolohte', 23494)
  , ('2014-01-06', 16, 'gbchhkcotf',  3966)
  , ('2014-01-07', 17, 'zfmbpvpzvu', 28159)
  , ('2014-01-07', 18, 'yauwzpaxtx',  8715)
  , ('2014-01-07', 19, 'uyqboqfgex', 10805)
  , ('2014-01-08', 20, 'hiqdkrzcpq',  3462)
  , ('2014-01-08', 21, 'zosbvlylpv', 13999)
  , ('2014-01-08', 22, 'bwfbchzgnl',  2299)
  , ('2014-01-09', 23, 'zzgauelgrt', 16475)
  , ('2014-01-09', 24, 'qrzfcwecge',  6469)
  , ('2014-01-10', 25, 'njbpsrvvcq', 16584)
  , ('2014-01-10', 26, 'cyxfgumkst', 11339)
;


""


앞선 테이블을 가지고 날짜별 매출을 집계해 보겠습니다. 

In [3]:
%%bigquery --project mygcpproject-340112
  -- 9.1 날짜별 매출과 평균 구매액을 집계하는 쿼리
SELECT
    dt
  , COUNT(*) AS purchase_count
  , SUM(purchase_amount) AS total_amount
  , AVG(purchase_amount) AS avg_amount
FROM sqldata.purchase_log
GROUP BY dt
ORDER BY dt
;

,dt,purchase_count,total_amount,avg_amount
0,2014-01-01,2,24516,12258.000000
1,2014-01-02,2,36049,18024.500000
2,2014-01-03,3,53029,17676.333333
3,2014-01-04,3,29299,9766.333333
4,2014-01-05,3,48256,16085.333333
5,2014-01-06,3,29440,9813.333333
6,2014-01-07,3,47679,15893.000000
7,2014-01-08,3,19760,6586.666667
8,2014-01-09,2,22944,11472.000000
9,2014-01-10,2,27923,13961.500000


다음으로 토, 일과 같은 주말에 매출이 변동하는지 알아보기 위해 이동평균을 사용하여 날짜별 추이를 살펴보겠습니다.

In [4]:
%%bigquery --project mygcpproject-340112
  -- 9.2 날짜별 매출과 7일 이동평균을 집계하는 쿼리
SELECT
    dt
  , SUM(purchase_amount) AS total_amount


  -- 최근 최대 7일 동안의 평균 계산하기
  , AVG(SUM(purchase_amount))
    OVER(ORDER BY dt ROWS BETWEEN 6 PRECEDING AND CURRENT ROW)
    AS seven_day_avg
  
  -- 최근 7일 동안의 평균을 확실하게 계산하기
  , CASE
      WHEN
        7 = COUNT(*)
        OVER(ORDER BY dt ROWS BETWEEN 6 PRECEDING AND CURRENT ROW)
      THEN
        AVG(SUM(purchase_amount))
        OVER(ORDER BY dt ROWS BETWEEN 6 PRECEDING AND CURRENT ROW)
    END
    AS seven_day_avg_strict
FROM sqldata.purchase_log
GROUP BY dt
ORDER BY dt
;

,dt,total_amount,seven_day_avg,seven_day_avg_strict
0,2014-01-01,24516,24516.000000,NaN
1,2014-01-02,36049,30282.500000,NaN
2,2014-01-03,53029,37864.666667,NaN
3,2014-01-04,29299,35723.250000,NaN
4,2014-01-05,48256,38229.800000,NaN
5,2014-01-06,29440,36764.833333,NaN
6,2014-01-07,47679,38324.000000,38324.000000
7,2014-01-08,19760,37644.571429,37644.571429
8,2014-01-09,22944,35772.428571,35772.428571
9,2014-01-10,27923,32185.857143,32185.857143


In [5]:
%%bigquery --project mygcpproject-340112
  -- 9.3 날짜별 매출과 당월 누계 매출을 집계하는 쿼리
SELECT
    dt
  -- '연, 월' 추출하기
  , substr(dt, 1, 7) AS year_month
  , SUM(purchase_amount) AS total_amount
  , SUM(SUM(purchase_amount))
    OVER(PARTITION BY substr(dt, 1, 7) ORDER BY dt ROWS UNBOUNDED PRECEDING)
  AS agg_amount
FROM
  sqldata.purchase_log
GROUP BY dt
ORDER BY dt
;

,dt,year_month,total_amount,agg_amount
0,2014-01-01,2014-01,24516,24516
1,2014-01-02,2014-01,36049,60565
2,2014-01-03,2014-01,53029,113594
3,2014-01-04,2014-01,29299,142893
4,2014-01-05,2014-01,48256,191149
5,2014-01-06,2014-01,29440,220589
6,2014-01-07,2014-01,47679,268268
7,2014-01-08,2014-01,19760,288028
8,2014-01-09,2014-01,22944,310972
9,2014-01-10,2014-01,27923,338895


앞선 쿼리에서는 날짜별 매출과 월별 누계 매출을 동시에 집계하기 위해 substring 함수를 이용하여 날짜에서 '연, 월' 부분을 추출했습니다. 이어서 GROUP BY 를 사용하여 날짜별로 집계한 합계 금액에 윈도 함수를 적용하여 날짜 순서대로 누계 매출을 계산합니다.  
앞선 쿼리에서 반복되어 나오는 SUM과 SUBSTR을 외부로 빼내 알아보기 쉽게 이름을 붙여보겠습니다. 

In [6]:
%%bigquery --project mygcpproject-340112
  -- 9.4 날짜별 매출을 일시 테이블로 만드는 쿼리
WITH
daily_purchase AS (
  SELECT
      dt
      -- '연', '월', '일'을 각각 추출하기
    , substr(dt, 1, 4) AS year
    , substr(dt, 6, 2) AS month
    , substr(dt, 9, 2) AS date
    , SUM(purchase_amount) AS purchase_amount
    , COUNT(order_id) AS orders
  FROM sqldata.purchase_log
  GROUP BY dt
)
SELECT
  *
FROM
  daily_purchase
ORDER BY dt
;

,dt,year,month,date,purchase_amount,orders
0,2014-01-01,2014,01,01,24516,2
1,2014-01-02,2014,01,02,36049,2
2,2014-01-03,2014,01,03,53029,3
3,2014-01-04,2014,01,04,29299,3
4,2014-01-05,2014,01,05,48256,3
5,2014-01-06,2014,01,06,29440,3
6,2014-01-07,2014,01,07,47679,3
7,2014-01-08,2014,01,08,19760,3
8,2014-01-09,2014,01,09,22944,2
9,2014-01-10,2014,01,10,27923,2


날짜를 분할하고 날짜별로 합계 금액을 계산하여 daily_purchase 테이블로 지정하였습니다.  
다음 쿼리는 daily_purchase 테이블을 사용하여 당월 매출 누계를 집계하는 쿼리입니다. 

In [7]:
%%bigquery --project mygcpproject-340112
  -- 9.5 daily_puchase 테이블에 대해 당월 누계 매출을 집계하는 쿼리
WITH
daily_purchase AS (
  SELECT
      dt
      -- '연', '월', '일'을 각각 추출하기
    , substr(dt, 1, 4) AS year
    , substr(dt, 6, 2) AS month
    , substr(dt, 9, 2) AS date
    , SUM(purchase_amount) AS purchase_amount
    , COUNT(order_id) AS orders
  FROM sqldata.purchase_log
  GROUP BY dt
)
SELECT
    dt
  , concat(year, '-', month) AS year_month
  , purchase_amount
  , SUM(purchase_amount)
    OVER(PARTITION BY year, month ORDER BY dt ROWS UNBOUNDED PRECEDING)
  AS agg_amount
FROM daily_purchase
ORDER BY dt
;

,dt,year_month,purchase_amount,agg_amount
0,2014-01-01,2014-01,24516,24516
1,2014-01-02,2014-01,36049,60565
2,2014-01-03,2014-01,53029,113594
3,2014-01-04,2014-01,29299,142893
4,2014-01-05,2014-01,48256,191149
5,2014-01-06,2014-01,29440,220589
6,2014-01-07,2014-01,47679,268268
7,2014-01-08,2014-01,19760,288028
8,2014-01-09,2014-01,22944,310972
9,2014-01-10,2014-01,27923,338895


In [8]:
%%bigquery --project mygcpproject-340112
DROP TABLE IF EXISTS sqldata.purchase_log;
CREATE TABLE sqldata.purchase_log(
    dt              string
  , order_id        integer
  , user_id         string
  , purchase_amount integer
);

INSERT INTO sqldata.purchase_log
VALUES
    ('2014-01-01',    1, 'rhwpvvitou', 13900)
  , ('2014-02-08',   95, 'chtanrqtzj', 28469)
  , ('2014-03-09',  168, 'bcqgtwxdgq', 18899)
  , ('2014-04-11',  250, 'kdjyplrxtk', 12394)
  , ('2014-05-11',  325, 'pgnjnnapsc',  2282)
  , ('2014-06-12',  400, 'iztgctnnlh', 10180)
  , ('2014-07-11',  475, 'eucjmxvjkj',  4027)
  , ('2014-08-10',  550, 'fqwvlvndef',  6243)
  , ('2014-09-10',  625, 'mhwhxfxrxq',  3832)
  , ('2014-10-11',  700, 'wyrgiyvaia',  6716)
  , ('2014-11-10',  775, 'cwpdvmhhwh', 16444)
  , ('2014-12-10',  850, 'eqeaqvixkf', 29199)
  , ('2015-01-09',  925, 'efmclayfnr', 22111)
  , ('2015-02-10', 1000, 'qnebafrkco', 11965)
  , ('2015-03-12', 1075, 'gsvqniykgx', 20215)
  , ('2015-04-12', 1150, 'ayzvjvnocm', 11792)
  , ('2015-05-13', 1225, 'knhevkibbp', 18087)
  , ('2015-06-10', 1291, 'wxhxmzqxuw', 18859)
  , ('2015-07-10', 1366, 'krrcpumtzb', 14919)
  , ('2015-08-08', 1441, 'lpglkecvsl', 12906)
  , ('2015-09-07', 1516, 'mgtlsfgfbj',  5696)
  , ('2015-10-07', 1591, 'trgjscaajt', 13398)
  , ('2015-11-06', 1666, 'ccfbjyeqrb',  6213)
  , ('2015-12-05', 1741, 'onooskbtzp', 26024)
;


""


앞서 만든 daily_purchase 테이블을 이용하여 JOIN을 사용하지 않고 2015년 매출의 작대비를 계산해보도록 하겠습니다. 

In [9]:
%%bigquery --project mygcpproject-340112
  -- 9.6 월별 매출과 작대비를 계산하는 쿼리
WITH
daily_purchase AS (
  SELECT
      dt
      -- '연', '월', '일'을 각각 추출하기
    , substr(dt, 1, 4) AS year
    , substr(dt, 6, 2) AS month
    , substr(dt, 9, 2) AS date
    , SUM(purchase_amount) AS purchase_amount
    , COUNT(order_id) AS orders
  FROM sqldata.purchase_log
  GROUP BY dt
)
SELECT
    month
  , SUM(CASE year WHEN '2014' THEN purchase_amount END) AS amount_2014
  , SUM(CASE year WHEN '2015' THEN purchase_amount END) AS amount_2015
  , 100.0
    * SUM(CASE year WHEN '2015' THEN purchase_amount END)
    / SUM(CASE year WHEN '2014' THEN purchase_amount END)
    AS rate
FROM
  daily_purchase
GROUP BY month
ORDER BY month
;

,month,amount_2014,amount_2015,rate
0,01,13900,22111,159.071942
1,02,28469,11965,42.028171
2,03,18899,20215,106.963331
3,04,12394,11792,95.142811
4,05,2282,18087,792.594216
5,06,10180,18859,185.255403
6,07,4027,14919,370.474298
7,08,6243,12906,206.727535
8,09,3832,5696,148.643006
9,10,6716,13398,199.493746


다음으로 z차트를 그려보겠습니다. Z차트에 필요한 지표는 월 단위로 집계하기 때문에, 구매 로그를 기반으로 월별 매출을 집계하고 각 월의 매출에 대해 누계 매출과 이동년계를 계산합니다. 

In [10]:
%%bigquery --project mygcpproject-340112
  -- 9.7 2015년 매출에 대한 Z 차트를 작성하는 쿼리
WITH
daily_purchase AS (
  SELECT
      dt
      -- '연', '월', '일'을 각각 추출하기
    , substr(dt, 1, 4) AS year
    , substr(dt, 6, 2) AS month
    , substr(dt, 9, 2) AS date
    , SUM(purchase_amount) AS purchase_amount
    , COUNT(order_id) AS orders
  FROM sqldata.purchase_log
  GROUP BY dt
)
, monthly_amount AS (
  -- 월별 매출 집계하기
  SELECT
      year
    , month
    , SUM(purchase_amount) AS amount
  FROM daily_purchase
  GROUP BY year, month
)
, calc_index AS (
SELECT
    year
  , month
  , amount
    -- 2015년 누계 매출 집계하기
  , SUM(CASE WHEN year = '2015' THEN amount END)
    OVER(ORDER BY year, month ROWS UNBOUNDED PRECEDING)
  AS agg_amount
    -- 당월부터 11개월 이전까지의 매출 합계(이동년계) 집계하기
  , SUM(amount)
    OVER(ORDER BY year, month ROWS BETWEEN 11 PRECEDING AND CURRENT ROW)
  AS year_avg_amount
FROM
  monthly_amount
ORDER BY
  year, month
)
  -- 마지막으로 2015년의 데이터만 압축하기
SELECT
  concat(year, '-', month) AS year_month
, amount
, agg_amount
, year_avg_amount
FROM
  calc_index
WHERE
  year = '2015'
ORDER BY
  year_month
;

,year_month,amount,agg_amount,year_avg_amount
0,2015-01,22111,22111,160796
1,2015-02,11965,34076,144292
2,2015-03,20215,54291,145608
3,2015-04,11792,66083,145006
4,2015-05,18087,84170,160811
5,2015-06,18859,103029,169490
6,2015-07,14919,117948,180382
7,2015-08,12906,130854,187045
8,2015-09,5696,136550,188909
9,2015-10,13398,149948,195591


단순한 매출 집계로는 매출 상승과 하락의 이유를 알 수 없기 때문에 매출 리포트를 만들 때에는 구매 횟수나 단가 등의 주변 데이터를 고려해야 합니다.  
  
다음으로 매출과 관련된 지표들을 함께 집계하는 쿼리를 작성해보겠습니다. 

In [13]:
%%bigquery --project mygcpproject-340112
  -- 9.8 매출과 관련된 지표를 집계하는 쿼리
WITH
daily_purchase AS (
  SELECT
      dt
    , substr(dt, 1, 4) AS year
    , substr(dt, 6, 2) AS month
    , substr(dt, 9, 2) AS date
    , SUM(purchase_amount) AS purchase_amount
    , COUNT(order_id) AS orders
  FROM sqldata.purchase_log
  GROUP BY dt
)
, monthly_purchase AS (
  SELECT
      year
    , month
    , SUM(orders) AS orders
    , AVG(purchase_amount) AS avg_amount
    , SUM(purchase_amount) AS monthly
  FROM daily_purchase
  GROUP BY year, month
)
SELECT
    concat(year, '-', month) AS year_month
  , orders
  , avg_amount
  , monthly
  , SUM(monthly)
      OVER(PARTITION BY year ORDER BY month ROWS UNBOUNDED PRECEDING)
    AS agg_amount
    -- 12개월 전의 매출 구하기
  , LAG(monthly, 12)
      OVER(ORDER BY year, month)
    AS last_year
    -- 12개월 전의 매출과 비교해서 비율 구하기
  , 100.0
    *monthly
    /LAG(monthly, 12)
      OVER(ORDER BY year, month)
    AS rate
FROM
  monthly_purchase
ORDER BY
  year_month
;


,year_month,orders,avg_amount,monthly,agg_amount,last_year,rate
0,2014-01,1,13900.0,13900,13900,NaN,NaN
1,2014-02,1,28469.0,28469,42369,NaN,NaN
2,2014-03,1,18899.0,18899,61268,NaN,NaN
3,2014-04,1,12394.0,12394,73662,NaN,NaN
4,2014-05,1,2282.0,2282,75944,NaN,NaN
5,2014-06,1,10180.0,10180,86124,NaN,NaN
6,2014-07,1,4027.0,4027,90151,NaN,NaN
7,2014-08,1,6243.0,6243,96394,NaN,NaN
8,2014-09,1,3832.0,3832,100226,NaN,NaN
9,2014-10,1,6716.0,6716,106942,NaN,NaN


purchase_log 데이터를 이용하여 월 단위 매출을 정리한 monthly_purchase를 만들고, 윈도함수를 적용하였습니다. LAG함수를 이용해 12개월 전의 매출을 검색했고, 데이터 내에서 찾을 수 없을 때 null값이 출력된 것을 볼 수 있습니다. 